                                                             
                                                                 
                                                                  
                                                                   
                                                                    
                                                                     
                                                                      Assignment No:10

Name:Shreya Desai

PRN:22510049

Batch:T3

In [ ]:

import pandas as pd
import numpy as np
import random


from sklearn.model_selection importso train_test_split

from sklearn.tree import DecisionTreeClassifier


from sklearn.linear_model import LogisticRegression


from sklearn.metrics import accuracy_score, recall_score


np.random.seed(42)


In [ ]:
def load_and_preprocess_data(filepath):


    # Load dataset
    data = pd.read_csv(filepath)
    print(f" Loaded data! Shape: {data.shape}")

    # Drop 'id' column
    data = data.drop(columns=['id'])
    print(" Dropped 'id' column.")

    # Check for missing values
    print("\n Checking for missing values:")
    print(data.isnull().sum().to_string())

    # Encode target column: Diagnosis (M = 1, B = 0)
    data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})
    print(" Encoded 'diagnosis' column: M -> 1, B -> 0")

    # Separate features and target
    X = data.drop(columns=['diagnosis'])
    y = data['diagnosis']
    print(f" Features shape: {X.shape}, Target shape: {y.shape}")

    return X, y


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

def skew_train_test_split(X, y, n_move=120, random_state=42):


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=random_state
    )

    # Combine X and y for easier row manipulation
    train_df = pd.concat([X_train, y_train], axis=1)
    test_df = pd.concat([X_test, y_test], axis=1)

    # Get rows where diagnosis is 1 (Malignant)
    m_class_rows = train_df[train_df['diagnosis'] == 1]

    # Randomly select rows to move
    to_move = m_class_rows.sample(n=n_move, random_state=random_state)

    # Remove selected malignant rows from train
    train_df = train_df.drop(to_move.index)

    # Add selected malignant rows to test
    test_df = pd.concat([test_df, to_move])

    print(f" Moved {n_move} 'M' (malignant) cases from train to test.")
    print(f" Final training size: {train_df.shape[0]}, Final test size: {test_df.shape[0]}")

    # Final split into X and y again
    X_train = train_df.drop(columns=['diagnosis'])
    y_train = train_df['diagnosis']
    X_test = test_df.drop(columns=['diagnosis'])
    y_test = test_df['diagnosis']

    return X_train, X_test, y_train, y_test


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def train_decision_trees(X_train, y_train, n_trees=10):



    trees = []
    accuracies = []
    feature_importances = []

    for i in range(n_trees):


        # Sample the training data with replacement
        indices = np.random.choice(X_train.index, size=len(X_train), replace=True)
        X_sample = X_train.loc[indices]
        y_sample = y_train.loc[indices]

        # Create and train the decision tree
        tree = DecisionTreeClassifier(
            max_features='sqrt',
            class_weight='balanced',
            random_state=i
        )
        tree.fit(X_sample, y_sample)

        # Accuracy on full training set
        train_preds = tree.predict(X_train)
        acc = accuracy_score(y_train, train_preds)

        print(f" Tree {i+1} trained. Accuracy: {acc:.4f}")

        # Save the model and info
        trees.append(tree)
        accuracies.append(acc)
        feature_importances.append(tree.feature_importances_)

    return trees, accuracies, feature_importances


In [ ]:
import numpy as np

def get_top_features_simple(importances, feature_names, top_n=10):


    # Convert to numpy array for easy averaging
    importances_array = np.array(importances)

    # Average over all trees
    avg_importance = np.mean(importances_array, axis=0)

    # Get top N indices
    top_indices = np.argsort(avg_importance)[::-1][:top_n]

    # Get corresponding feature names
    top_features = [feature_names[i] for i in top_indices]

    print(f"\n Top important feature:")
    for i, feature in enumerate(top_features):
        print(f"{i+1}. {feature}")


    return top_feature


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def retrain_decision_trees_on_selected_features(X_train_selected, y_train, n_trees=10):

    print(f"\n Retraining {n_trees} decision trees using only selected features...")

    trees = []
    accuracies = []

    for i in range(n_trees):


        # Bootstrap sampling with replacement
        indices = np.random.choice(X_train_selected.index, size=len(X_train_selected), replace=True)
        X_sample = X_train_selected.loc[indices]
        y_sample = y_train.loc[indices]

        # Create and train tree
        tree = DecisionTreeClassifier(
            max_features='sqrt',
            class_weight='balanced',
            random_state=100 + i
        )
        tree.fit(X_sample, y_sample)

        # Accuracy on full training set
        preds = tree.predict(X_train_selected)
        acc = accuracy_score(y_train, preds)

        print(f" Retrained Tree {i+1} Accuracy: {acc:.4f}")

        trees.append(tree)
        accuracies.append(acc)

    return trees, accuracies


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

def train_final_models(retrained_trees, X_train_selected, y_train):


    # Get predictions of 10 trees on training data
    tree_preds = []

    for i, tree in enumerate(retrained_trees):
        preds = tree.predict(X_train_selected)
        tree_preds.append(preds)


    # Stack predictions as extra features (shape: n_samples, 10)
    tree_preds_matrix = np.column_stack(tree_preds)

    # Combine: top features + tree predictions
    import pandas as pd
    X_combined = pd.concat([X_train_selected.reset_index(drop=True),
                            pd.DataFrame(tree_preds_matrix, columns=[f"tree_{i+1}" for i in range(len(retrained_trees))])],
                            axis=1)

    # Model 1: Logistic Regression
    log_model = LogisticRegression(class_weight='balanced', max_iter=1000)
    log_model.fit(X_combined, y_train)


    # Model 2: Master Decision Tree
    master_tree = DecisionTreeClassifier(class_weight='balanced', max_features='sqrt', random_state=200)
    master_tree.fit(X_combined, y_train)


    return log_model, master_tree


In [ ]:
from sklearn.metrics import accuracy_score, recall_score
import pandas as pd
import numpy as np

def evaluate_final_models(retrained_trees, log_model, master_tree, X_test_selected, y_test):


    #  Get predictions from each retrained tree on test set
    test_tree_preds = []
    for i, tree in enumerate(retrained_trees):
        preds = tree.predict(X_test_selected)
        test_tree_preds.append(preds)


    #  Stack predictions horizontally
    test_preds_matrix = np.column_stack(test_tree_preds)

    #  Combine selected features  tree predictions
    X_test_combined = pd.concat([X_test_selected.reset_index(drop=True),
                                 pd.DataFrame(test_preds_matrix, columns=[f"tree_{i+1}" for i in range(len(retrained_trees))])],
                                 axis=1)

    #  Make predictions using logistic regression
    y_pred_log = log_model.predict(X_test_combined)
    acc_log = accuracy_score(y_test, y_pred_log)
    # Make predictions using master decision tree
    y_pred_tree = master_tree.predict(X_test_combined)
    acc_tree = accuracy_score(y_test, y_pred_tree)


    #  Print results
    print("\n Final Model Evaluation on Test Data:")
    print(f" Logistic Regression Accuracy: {acc_log:.4f} ")
    print(f" Master Decision Tree Accuracy: {acc_tree:.4f}")

    return acc_log, recall_log, acc_tree, recall_tree


In [ ]:
def main_pipeline():


    # Step 1: Load and preprocess data
    X, y = load_and_preprocess_data('/content/Cancer_Data.csv')

    # Step 2: Split and skew the data
    X_train, X_test, y_train, y_test = skew_train_test_split(X, y)


    # Step 3: Train 10 decision trees with sample & feature bagging
    trees, importances, accuracies = train_decision_trees(X_train, y_train, n_trees=10)

    # Step 4: Combine feature importances and select top features
    top_features = get_top_features_simple(importances, X_train.columns, top_n=10)

# Now select only those features from X_train and X_test
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]


    # Step 5: Retrain trees using only selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]
    retrained_trees, retrained_accuracies = retrain_decision_trees_on_selected_features(X_train_selected, y_train)

    # Step 6: Train final logistic model and master decision tree
    log_model, master_tree = train_final_models(retrained_trees, X_train_selected, y_train)

    # Step 7: Evaluate on test data
    evaluate_final_models(retrained_trees, log_model, master_tree, X_test_selected, y_test)



main_pipeline()



 Loading and preprocessing dataset...
 Loaded data! Shape: (569, 33)
 Dropped 'id' column.

 Checking for missing values:
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave p

Observation:
1)The 10 decision trees worked well, with accuracies between 97% and 99%.


2)After using fewer features, the accuracy of the decision trees dropped a little, from 94% to 96%.


3)Logistic regression had an accuracy of 74.8% and a recall of 67.3%, meaning it did well at identifying cancer cases.

4)he master decision tree had an accuracy of 73.9% and recall of 65.4%, which is lower than logistic regression.

5)The logistic regression model did better than the decision trees and the master decision tree in terms of both accuracy and recall.